In [22]:
from neo4j import GraphDatabase
import json
from time import time


uri = "neo4j://127.0.0.1:7687"
username = "neo4j"
password = "12345678"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [23]:
# === 1. Tạo constraint ===
with driver.session(database="traffic-england-3") as session:
    session.run("""
    CREATE CONSTRAINT intersection_id_unique IF NOT EXISTS
    FOR (i:Intersection) REQUIRE i.id IS UNIQUE
    """)
print("✅ Constraint đã có (hoặc đã tồn tại).")

✅ Constraint đã có (hoặc đã tồn tại).


In [11]:
with open("D:/ALL_Python/EXAMPLE_DOAN/MODEL_BIGDATA/data/du_lieu.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [24]:
# --- 4. Chuẩn bị dữ liệu Intersections ---
def _prepare_intersection(rec):
    return {
        "id": rec.get("count_point_id"),
        "name": rec.get("start_junction_road_name"),
        "latitude": rec.get("latitude"),
        "longitude": rec.get("longitude"),
        "region_name": rec.get("region_name"),
    }

rows = [_prepare_intersection(r) for r in data if r.get("count_point_id") is not None]

In [25]:
# --- 5. Tạo node Intersection và Region ---
create_intersections = """
UNWIND $batch AS row
MERGE (i:Intersection {id: row.id})
SET i.name = row.name,
    i.latitude = row.latitude,
    i.longitude = row.longitude

WITH i, row
MERGE (r:Region {name: row.region_name})
MERGE (r)-[:HAS_INTERSECTION]->(i)
"""

In [27]:
print("🚧 Import Intersections và Regions ...")
start = time()
BATCH_SIZE = 2000

with driver.session(database="traffic-england-3") as session:
    total = len(rows)
    for i in range(0, total, BATCH_SIZE):
        batch = rows[i:i+BATCH_SIZE]
        session.run(create_intersections, batch=batch)
        print(f"✅ Imported {min(i+BATCH_SIZE, total)}/{total} Intersections")
end = time()
print(f"⏱️ Hoàn tất tạo Intersections và Regions trong {end-start:.1f}s")

🚧 Import Intersections và Regions ...
✅ Imported 2000/4337136 Intersections
✅ Imported 4000/4337136 Intersections
✅ Imported 6000/4337136 Intersections
✅ Imported 8000/4337136 Intersections
✅ Imported 10000/4337136 Intersections
✅ Imported 12000/4337136 Intersections
✅ Imported 14000/4337136 Intersections
✅ Imported 16000/4337136 Intersections
✅ Imported 18000/4337136 Intersections
✅ Imported 20000/4337136 Intersections
✅ Imported 22000/4337136 Intersections
✅ Imported 24000/4337136 Intersections
✅ Imported 26000/4337136 Intersections
✅ Imported 28000/4337136 Intersections
✅ Imported 30000/4337136 Intersections
✅ Imported 32000/4337136 Intersections
✅ Imported 34000/4337136 Intersections
✅ Imported 36000/4337136 Intersections
✅ Imported 38000/4337136 Intersections
✅ Imported 40000/4337136 Intersections
✅ Imported 42000/4337136 Intersections
✅ Imported 44000/4337136 Intersections
✅ Imported 46000/4337136 Intersections
✅ Imported 48000/4337136 Intersections
✅ Imported 50000/4337136 Inter

In [28]:
with driver.session(database="traffic-england-3") as session:
    session.run("CREATE INDEX intersection_id_index IF NOT EXISTS FOR (i:Intersection) ON (i.id)")
    session.run("CREATE INDEX intersection_name_index IF NOT EXISTS FOR (i:Intersection) ON (i.name)")
    session.run("CREATE INDEX road_name_index IF NOT EXISTS FOR (r:Road) ON (r.name)")
    session.run("CREATE INDEX region_name_index IF NOT EXISTS FOR (r:Region) ON (r.name)")
print("✅ Đã tạo index để tăng tốc MATCH.")


✅ Đã tạo index để tăng tốc MATCH.


In [29]:
# --- 3. Chuẩn bị dữ liệu ---
rows = []
for r in data:
    if r.get("count_point_id") and r.get("end_junction_road_name"):
        rows.append({
            "road_name": r.get("road_name"),
            "road_type": r.get("road_type"),
            "start_id": r.get("count_point_id"),  # ID nút giao đầu
            "end_name": r.get("end_junction_road_name"),  # Tên nút giao cuối (tạm thời)
            "region": r.get("region_name"),
            "length_km": r.get("link_length_km"),
            "latitude": r.get("latitude"),
            "longitude": r.get("longitude")
        })

print(f"📊 Tổng số quan hệ ROAD cần tạo: {len(rows)}")

📊 Tổng số quan hệ ROAD cần tạo: 1968540


In [ ]:
# # --- 3. Chuẩn bị dữ liệu ---
# # Tạo map để tìm id theo tên nút giao
# name_to_id = {}
# for r in data:
#     if r.get("start_junction_road_name") and r.get("count_point_id"):
#         name_to_id[r["start_junction_road_name"]] = r["count_point_id"]

# rows = []
# for r in data:
#     start_id = r.get("count_point_id")
#     end_name = r.get("end_junction_road_name")
#     end_id = name_to_id.get(end_name)  # tra ID tương ứng nếu có

#     if start_id and end_id:
#         rows.append({
#             "road_name": r.get("road_name"),
#             "road_type": r.get("road_type"),
#             "start_id": start_id,
#             "end_id": end_id,
#             "region": r.get("region_name"),
#             "length_km": r.get("link_length_km"),
#         })

# print(f"📊 Tổng số quan hệ ROAD cần tạo: {len(rows)}")


📊 Tổng số quan hệ ROAD cần tạo: 1610352


In [30]:
# --- 4. Cypher để tạo node và quan hệ ---
create_road_query = """
UNWIND $batch AS row
MATCH (s:Intersection {id: row.start_id})
MATCH (e:Intersection {name: row.end_name})
WITH s, e, row
MERGE (r:Road {name: row.road_name})
SET r.road_type = row.road_type,
    r.length_km = row.length_km

MERGE (reg:Region {name: row.region})
MERGE (reg)-[:HAS_ROAD]->(r)

MERGE (s)-[:CONNECTS_VIA]->(r)
MERGE (r)-[:CONNECTS_TO]->(e)
"""

In [ ]:
# create_road_query = """
# UNWIND $batch AS row
# MATCH (s:Intersection {id: row.start_id})
# MATCH (e:Intersection {id: row.end_id})
# WITH s, e, row
# MERGE (r:Road {name: row.road_name})
# SET r.road_type = row.road_type,
#     r.length_km = row.length_km
# MERGE (reg:Region {name: row.region})
# MERGE (reg)-[:HAS_ROAD]->(r)
# MERGE (s)-[:CONNECTS_VIA]->(r)
# MERGE (r)-[:CONNECTS_TO]->(e)
# """


In [ ]:
# # --- 5. Import dữ liệu ---
# start = time()
# BATCH_SIZE = 2000
# with driver.session(database="traffic-england-1") as session:
#     total = len(rows)
#     for i in range(0, total, BATCH_SIZE):
#         batch = rows[i:i+BATCH_SIZE]
#         session.run(create_road_query, batch=batch)
#         print(f"✅ Imported {min(i+BATCH_SIZE, total)}/{total} roads")

# end = time()
# print(f"⏱️ Hoàn tất tạo quan hệ ROAD trong {end-start:.1f}s")

In [32]:
def import_batch(tx, batch):
    tx.run(create_road_query, batch=batch)

BATCH_SIZE = 5000  # tăng lên vì write_transaction ổn định hơn
start = time()
total = len(rows)

with driver.session(database="traffic-england-3") as session:
    for i in range(0, total, BATCH_SIZE):
        batch = rows[i:i+BATCH_SIZE]
        session.execute_write(import_batch, batch)
        print(f"✅ Imported {min(i+BATCH_SIZE, total)}/{total}")

print(f"⏱️ Hoàn tất tạo quan hệ ROAD trong {time()-start:.1f}s")


✅ Imported 5000/1968540
✅ Imported 10000/1968540
✅ Imported 15000/1968540
✅ Imported 20000/1968540
✅ Imported 25000/1968540
✅ Imported 30000/1968540
✅ Imported 35000/1968540
✅ Imported 40000/1968540
✅ Imported 45000/1968540
✅ Imported 50000/1968540
✅ Imported 55000/1968540
✅ Imported 60000/1968540
✅ Imported 65000/1968540
✅ Imported 70000/1968540
✅ Imported 75000/1968540
✅ Imported 80000/1968540
✅ Imported 85000/1968540
✅ Imported 90000/1968540
✅ Imported 95000/1968540
✅ Imported 100000/1968540
✅ Imported 105000/1968540
✅ Imported 110000/1968540
✅ Imported 115000/1968540
✅ Imported 120000/1968540
✅ Imported 125000/1968540
✅ Imported 130000/1968540
✅ Imported 135000/1968540
✅ Imported 140000/1968540
✅ Imported 145000/1968540
✅ Imported 150000/1968540
✅ Imported 155000/1968540
✅ Imported 160000/1968540
✅ Imported 165000/1968540
✅ Imported 170000/1968540
✅ Imported 175000/1968540
✅ Imported 180000/1968540
✅ Imported 185000/1968540
✅ Imported 190000/1968540
✅ Imported 195000/1968540
✅ Impo

In [ ]:
# def import_road_batch(tx, batch):
#     tx.run(create_road_query, batch=batch)

# start = time()
# BATCH_SIZE = 1000

# with driver.session(database="traffic-england-2") as session:
#     total = len(rows)
#     for i in range(0, total, BATCH_SIZE):
#         batch = rows[i:i+BATCH_SIZE]
#         session.execute_write(import_road_batch, batch)
#         print(f"✅ Imported {min(i+BATCH_SIZE, total)}/{total} roads")

# end = time()
# print(f"⏱️ Hoàn tất tạo quan hệ ROAD trong {end-start:.1f}s")


✅ Imported 1000/1610352 roads
✅ Imported 2000/1610352 roads
✅ Imported 3000/1610352 roads
✅ Imported 4000/1610352 roads
✅ Imported 5000/1610352 roads
✅ Imported 6000/1610352 roads
✅ Imported 7000/1610352 roads
✅ Imported 8000/1610352 roads
✅ Imported 9000/1610352 roads
✅ Imported 10000/1610352 roads
✅ Imported 11000/1610352 roads
✅ Imported 12000/1610352 roads
✅ Imported 13000/1610352 roads
✅ Imported 14000/1610352 roads
✅ Imported 15000/1610352 roads
✅ Imported 16000/1610352 roads
✅ Imported 17000/1610352 roads
✅ Imported 18000/1610352 roads
✅ Imported 19000/1610352 roads
✅ Imported 20000/1610352 roads
✅ Imported 21000/1610352 roads
✅ Imported 22000/1610352 roads
✅ Imported 23000/1610352 roads
✅ Imported 24000/1610352 roads
✅ Imported 25000/1610352 roads
✅ Imported 26000/1610352 roads
✅ Imported 27000/1610352 roads
✅ Imported 28000/1610352 roads
✅ Imported 29000/1610352 roads
✅ Imported 30000/1610352 roads
✅ Imported 31000/1610352 roads
✅ Imported 32000/1610352 roads
✅ Imported 33000/